# Movie Genre Predictor

In [1]:
#import libraries 
import pandas as pd # data preprocessing 
import numpy as np # linear algebra 
import kagglehub # data importing 

import nltk # tokenizing 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

import contractions 

nltk.download('stopwords') 
nltk.download('punkt_tab')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))



c:\Users\shery\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shery\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\shery\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shery\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Download latest version
path = kagglehub.dataset_download("guru001/movie-genre-prediction")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\shery\.cache\kagglehub\datasets\guru001\movie-genre-prediction\versions\1


In [3]:
# Read the dataset, drop the id column 
df = pd.read_csv('./data/test.csv')
df.drop(columns="id")

,movie_name,synopsis,genre
0,A Death Sentence,"12 y.o. Ida's dad'll die without a DKK1,500,00...",action
1,Intermedio,A group of four teenage friends become trapped...,action
2,30 Chua Phai Tet,A guy left his home for 12 years till he came ...,action
3,Paranoiac,A man long believed dead returns to the family...,action
4,Ordinary Happiness,"After a deadly accident, Paolo comes back on E...",action
...,...,...,...
35995,Mysterious Doctor Satan,A mad scientist plots the domination of Americ...,action
35996,Blondes at Work,Steve promises his captain not to favor Torchy...,action
35997,The Hurricane,A Polynesian sailor unjustly imprisoned after ...,action
35998,Malayankunju,Anikkuttan is an ill-tempered electronics tech...,action


In [4]:
# Remove punctuation 
def remove_punctuation(text): 
    punctuation =  ",./;[]-=\\~!@#$%^&*()_+{}|\":?><"
    cleaned = "".join([c for c in text if c not in punctuation])
    return cleaned 

df["synopsis"] = df["synopsis"].apply(lambda x: remove_punctuation(x))
df.head(10)

,id,movie_name,synopsis,genre
0,16863,A Death Sentence,12 yo Ida's dad'll die without a DKK1500000 op...,action
1,48456,Intermedio,A group of four teenage friends become trapped...,action
2,41383,30 Chua Phai Tet,A guy left his home for 12 years till he came ...,action
3,84007,Paranoiac,A man long believed dead returns to the family...,action
4,40269,Ordinary Happiness,After a deadly accident Paolo comes back on Ea...,action
5,16524,The Syndicate,An alcoholic his wife a down and out American ...,action
6,21245,12 Depths,Struck by amnesia and framed for murder a man ...,action
7,84681,The Feud,A mysterious man returns to his home in Iowa f...,action
8,73352,Cabin Fever: Patient Zero,When a group of friends enjoying a bachelor cr...,action
9,37966,Peregrina,After a near death experience a young woman be...,action


In [ ]:
#tokenize text & remove contractions 
def tokenize(text):
    text = contractions.fix(text)
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.lower() not in stop_words]
    return tokens

df["synopsis"] = df["synopsis"].apply(tokenize)

df["synopsis"].head(10)


0    [12, yo, ida, 's, dad, 'll, die, without, dkk1...
1    [group, four, teenage, friends, become, trappe...
2    [guy, left, home, 12, years, till, came, back,...
3    [man, long, believed, dead, returns, family, e...
4    [deadly, accident, paolo, comes, back, earth, ...
5    [alcoholic, wife, american, pilot, young, germ...
6    [struck, amnesia, framed, murder, man, battles...
7    [mysterious, man, returns, home, iowa, brother...
8    [group, friends, enjoying, bachelor, cruise, c...
9    [near, death, experience, young, woman, begins...
Name: synopsis, dtype: object

In [ ]:
#lemmatize -> finds true base word 
def lemmatize(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text 

df["synopsis"] = df["synopsis"].apply(lambda x : lemmatize(x))
df.head(10)

,id,movie_name,synopsis,genre
0,16863,A Death Sentence,"[12, yo, ida, 's, dad, 'll, die, without, dkk1...",action
1,48456,Intermedio,"[group, four, teenage, friend, become, trapped...",action
2,41383,30 Chua Phai Tet,"[guy, left, home, 12, year, till, came, back, ...",action
3,84007,Paranoiac,"[man, long, believed, dead, return, family, es...",action
4,40269,Ordinary Happiness,"[deadly, accident, paolo, come, back, earth, 9...",action
5,16524,The Syndicate,"[alcoholic, wife, american, pilot, young, germ...",action
6,21245,12 Depths,"[struck, amnesia, framed, murder, man, battle,...",action
7,84681,The Feud,"[mysterious, man, return, home, iowa, brother,...",action
8,73352,Cabin Fever: Patient Zero,"[group, friend, enjoying, bachelor, cruise, ca...",action
9,37966,Peregrina,"[near, death, experience, young, woman, begin,...",action
